In [122]:
import pandas as pd
import numpy as np
import re
import string
import pickle

In [123]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation,'')
    return text

In [133]:
with open('../static/model/model.pickle','rb') as f:
    model=pickle.load(f)


In [125]:
with open('../static/model/corpora/stopwords/english','r') as file:
    sw=file.read().splitlines()

In [126]:
vocab=pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens=vocab[0].tolist()

In [127]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [128]:
def preprossing(text):
    data=pd.DataFrame([text],columns=['tweet'])
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    data['tweet']=data['tweet'].apply(lambda x:" ".join(re.sub(r'https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data['tweet']=data['tweet'].apply(remove_punctuations)
    data['tweet']=data['tweet'].str.replace('\d+','',regex=True)
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split() if x not in sw))
    data['tweet']=data['tweet'].apply(lambda x:" ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

<>:6: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
<>:6: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
C:\Users\Nilanka\AppData\Local\Temp\ipykernel_14460\3073540848.py:6: SyntaxWarning: "\d" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\d"? A raw string is also an option.
  data['tweet']=data['tweet'].str.replace('\d+','',regex=True)


In [ ]:
def vectorizer(ds,vocabulary):
    vectorized_list=[]

    for sentence in ds:
        sentence_list=np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i]=1

        vectorized_list.append(sentence_list)

    vectorized_list_new=np.asarray(vectorized_list,dtype=np.float32)

    return vectorized_list_new

In [130]:
def get_prediction(vectorized_text):
    prediction=model.predict(vectorized_text)
    if prediction[0]==1:
        return 'Negative'
    else:
        return 'Positive'

In [134]:
text='good product. I love it'
preprossed_text=preprossing(text)
vectorized_text=vectorizer(preprossed_text,tokens)
prediction=get_prediction(vectorized_text)
prediction

'Positive'